In [2]:
#!pip install torch==1.6.0
!pip install transformers==3.3.1
!pip install seqeval
!pip install fastprogress
!pip install attrdict

     |████████████████████████████████| 1.1MB 7.5MB/s 
     |████████████████████████████████| 890kB 12.3MB/s 
     |████████████████████████████████| 3.0MB 37.5MB/s 
     |████████████████████████████████| 1.2MB 60.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=cf47f24646e0c676133c2b6944ef069aee02dee745053093090c72870a8b0f31
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 51kB 4.7MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=d2ac0f6a0da4d6a4cdfc668732d788bd15d6c76e791edd26ab3952382db36fe3
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


해당 과제의 코드는 박장원님의 "https://github.com/monologg/KoELECTRA/tree/master/finetune" 레포지토리의 코드를 변경하여 만들어졌습니다.

추신. 좋은 코드 만들어주시고, 코드 사용을 흔쾌히 허락해주신 박장원님께 감사인사 드립니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
cd /content/drive/MyDrive/Colab\ Notebooks/WEEK_NLP_day20_assignment

/content/drive/MyDrive/Colab Notebooks/WEEK_NLP_day20_assignment


학습을 시작하기 전에 같이 동봉된 "preprocessing.ipynb"를 실행하여 나오는 "train.tsv","dev.tsv"를 "data/custom-ner"에 넣어줍니다.

In [5]:
!python3 run_ner.py --task custom-ner --config_file koelectra-small-v3.json # config/custom-ner 에서 원하는 모델의 config를 변경하고 넣어주세요.

2021-02-21 07:58:26.844484: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTok

In [8]:
from transformers import ElectraTokenizer, ElectraForTokenClassification  #다른 모델을 쓸 때는 tokenizer(ex.ElectraTokenizer)와 model(ex.ElectraForTokenClassification)를 바꿔주어야 합니다.

  
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")  #tokenizer를 바꾸었다면 from_pretrained에 들어가는 부분을 바꿔주어야 합니다. model config에 있는 model 주소를 적어주시면 됩니다. ex. monologg/koelectra-base-v3-discriminator

model = ElectraForTokenClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/WEEK_NLP_day20_assignment/ckpt/koelectra-small-v3-custom-ner-ckpt/checkpoint-4000")  # checkpoint가 있는 디렉토리 기입


In [9]:
label_list=['DT-B','DT-I','LC-B','LC-I','OG-B','OG-I','PS-B','PS-I','QT-B','QT-I','TI-B','TI-I','O']

Kaggle Competition page에서 "test.txt"를 다운 받아서 "data/custom-ner"에 넣어줍니다.

In [10]:
texts=[]
with open("data/custom-ner/test.txt") as f:
  for line in f:
    texts.append(line.rsplit("\n")[0])

In [11]:
import torch
from tqdm.auto import tqdm

In [12]:
labels=[]
for idx, sequence in tqdm(enumerate(texts)):
  tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
  inputs = tokenizer.encode(sequence, return_tensors="pt")
  outputs = model(inputs)
  predictions = torch.argmax(outputs[0], dim=2)
  labels_=[]
  for i,j in zip(tokens,predictions[0].tolist()):
    if i in ['[CLS]', '[SEP]']:
      continue
    if "##" in i:
      continue
    labels_.append(label_list[j])
  labels.extend(labels_)


"submission.csv"를 kaggle competition에 제출합니다.

데이터 전처리,모델 config 튜닝을 이용하여 더 좋은 성적을 얻어보세요.

In [13]:
with open("submission_2.csv","w") as f:
  f.write("id,tag\n")
  for idx,tag in enumerate(labels):
    f.write(str(idx))
    f.write(",")
    f.write(tag)
    f.write("\n")